In [1]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
from haversine import haversine
from tqdm.notebook import tqdm
import numpy as np

In [12]:
from sklearn.preprocessing import LabelEncoder

In [2]:
endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
schema = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema)
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

def load_data_from_rds(tabel_name):
  sql = "SELECT * FROM {}".format(tabel_name)
  df = pd.read_sql(sql, db_connection)
  return df

In [8]:
df = load_data_from_rds('Re_Main_승현')
df

,번호,생태통로명,주소,도로명,도로번호,차선_수,위도_도,위도_분,위도_초,경도_도,...,생태통로좌표(경도),모니터링기간,수행기관,수행자,모니터링방법,등산객이용빈도,모니터링신뢰도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",개선필요사항
0,1495,가계통로,경상남도 산청군 생비량면 가계리 산279-3,일반국도,20호선,왕복4차선,35,21.0,36.680000,128,...,경도 128도 3분 49.5초,2017.07.17. ~ 2017.07.28.,진주국토관리사무소,박상훈,None,극히 적음,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,판단 불가,None,None
1,1277,가대리,충청북도 단양군 가곡면 가대리,국도,59호선,왕복2차선,37,2.0,2.700000,128,...,경도 128도 24분 23.8초,2016.01.01. ~ 2016.09.20.,도로관리사업소,박찬일,None,None,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,판단 불가,None,None
2,1278,가산리,충청북도 단양군 단성면 가산리,국도,59호선,왕복2차선,36,53.0,53.957576,128,...,경도 128도 18분 20.275초,2016.01.01. ~ 2016.09.20.,도로관리사업소,박찬일,None,None,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,판단 불가,종명: 고라니 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,None
3,1279,가산리2,충청북도 단양군 단성면 가산리,국도,59호선,왕복2차선,36,53.0,48.718150,128,...,경도 128도 18분 20.917초,2016.01.01. ~ 2016.09.20.,도로관리사업소,박찬일,None,None,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,판단 불가,종명: 고라니 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,None
4,1090,각리,충청북도 청원군 오창읍 각리,지방도,540호,왕복6차선,36,42.0,48.100000,127,...,경도 127도 25분 19.2초,2016.09.01. ~ 2016.09.30.,충북도로관리사업소,문민규,None,매일 다수,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,보통(주 35회 야생동물 이용),종명: 개 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: 고양...,#NAME?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,1089,None,충청북도 청주시 청원구 오창읍 각리,과학산업2로,구룡리 472도,왕복6차선,36,42.0,46.000000,127,...,경도 127도 25분 14.0초,2016.09.26. ~ 2016.09.30.,청원구청,박태동 외 1인,None,None,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,판단 불가,종명: 고라니 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,#NAME?
508,1087,None,충청북도 청주시 청원구 오창읍 구룡리,오창공원로,구룡리 442도,왕복4차선,36,43.0,2.000000,127,...,경도 127도 25분 4.0초,2016.09.26. ~ 2016.09.30.,청원구,박태동 외 1인,None,매일 다수,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,판단 불가,종명: 고라니 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,#NAME?
509,1088,None,충청북도 청주시 청원구 오창읍 양청리,오창공원로,양청리 882도(호수공원 주변),왕복4차선,36,42.0,44.000000,127,...,경도 127도 25분 31.0초,2016.09.26. ~ 2016.09.30.,청원구청,박태동 외 1인,None,매일 다수,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,판단 불가,종명: 고라니 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,#NAME?
510,1091,None,충청북도 청주시 청원구 오창읍 여천리,리도,213호선,왕복2차선,36,46.0,3.000000,127,...,경도 127도 28분 24.0초,2016.09.26. ~ 2016.09.30.,청원구청,박태동 외 1인,None,None,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,판단 불가,종명: 멧돼지 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,#NAME?


In [10]:
col = list(df.columns)
col

['번호',
 '생태통로명',
 '주소',
 '도로명',
 '도로번호',
 '차선_수',
 '위도_도',
 '위도_분',
 '위도_초',
 '경도_도',
 '경도_분',
 '경도_초',
 '위도_도분초',
 '경도_도분초',
 '위도_십진법',
 '경도_십진법',
 'num',
 '도로',
 '관리기관',
 '설치기관',
 '생태통로타입',
 '준공년도',
 '생태통로좌표(위도)',
 '생태통로좌표(경도)',
 '모니터링기간',
 '수행기관',
 '수행자',
 '모니터링방법',
 '등산객이용빈도',
 '모니터링신뢰도',
 '생태통로효율성',
 '이용확인종,이용빈도높은종,멸종위기종이용빈도',
 '개선필요사항']

# 생태통로타입

In [28]:
df.isnull().sum()

번호                           0
생태통로명                      126
주소                           0
도로명                          9
도로번호                        21
차선_수                         0
위도_도                         0
위도_분                         1
위도_초                         0
경도_도                         0
경도_분                         0
경도_초                         0
위도_도분초                       0
경도_도분초                       0
위도_십진법                       0
경도_십진법                       0
num                         13
도로                          14
관리기관                        13
설치기관                        13
생태통로타입                      13
준공년도                        16
생태통로좌표(위도)                  13
생태통로좌표(경도)                  13
모니터링기간                     104
수행기관                       137
수행자                        106
모니터링방법                     354
등산객이용빈도                    285
모니터링신뢰도                    106
생태통로효율성                     94
이용확인종,이용빈도높은종,멸종위기종이용빈도    301
개선필요사항  

In [11]:
df['생태통로타입'].unique()

array(['육교형', '터널형', None, '양서·파충류 통로'], dtype=object)

In [29]:
df[df['생태통로타입'].isnull()]

,번호,생태통로명,주소,도로명,도로번호,차선_수,위도_도,위도_분,위도_초,경도_도,...,생태통로좌표(경도),모니터링기간,수행기관,수행자,모니터링방법,등산객이용빈도,모니터링신뢰도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",개선필요사항
26,1533,고암 생태통로,경기도 양주시 고암동 산44-3,국대도,3호선,왕복6차선,37,49.0,36.00,127,...,None,None,None,None,None,None,None,None,None,None
101,1521,동해선 117.93k(노학터널),강원도 속초시 노학동,동해선,65,왕복4차선,38,10.0,48.50,128,...,None,None,None,None,None,None,None,None,None,None
200,1345,수로암거,경상북도 봉화군 소천면 고선리 산61-22,국도,31호선,왕복2차선,37,0.0,3.92,128,...,None,None,None,None,None,None,None,None,None,None
259,1543,용화생태통로,경상북도 상주시 화북면 운흥리,국도,37,왕복2차선,36,35.0,40.50,127,...,None,None,None,None,None,None,None,None,None,None
287,1479,입암,전라북도 정읍시 입암면 신면리 산18-1,국도,1호선,왕복4차선,35,31.0,6.27,126,...,None,None,None,None,None,None,None,None,None,None
335,1030,청계산 생태통로,경기도 양평군 양서면 목왕리,군도,1호선,왕복2차선,37,34.0,208.00,127,...,None,None,None,None,None,None,None,None,None,None
389,1548,None,강원도 인제군 상남면 상남리 1627-5,고속도로,60,왕복4차선,37,50.0,3.60,128,...,None,None,None,None,None,None,None,None,None,None
390,1549,None,강원도 인제군 상남면 상남리 1649-5,고속,60,왕복4차선,37,49.0,53.10,128,...,None,None,None,None,None,None,None,None,None,None
405,1551,None,경기도 안성시 삼죽면 진촌리,국도,38,왕복2차선,37,4.0,4.50,127,...,None,None,None,None,None,None,None,None,None,None
425,1544,None,경상남도 진주시 진성면 구천리,국도,2,왕복4차선,35,10.0,8.70,128,...,None,None,None,None,None,None,None,None,None,None


In [27]:
df['생태통로효율성'].unique()

array(['판단 불가', '보통(주 35회 야생동물 이용)', '높음(거의 매일 야생동물 이용)', None,
       '낮음(주 2회 이하 야생동물 이용)'], dtype=object)

In [40]:
df[df['생태통로효율성']=='높음(거의 매일 야생동물 이용)']

,번호,생태통로명,주소,도로명,도로번호,차선_수,위도_도,위도_분,위도_초,경도_도,...,생태통로좌표(경도),모니터링기간,수행기관,수행자,모니터링방법,등산객이용빈도,모니터링신뢰도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",개선필요사항
5,1023,갈마치생태통로,경기도 성남시 중원구 갈현동,None,389,왕복2차선,37,25.0,11.000000,127,...,경도 127도 11분 15.0초,2022.05.02. ~ 2022.06.03.,성남시,성남시 자연환경모니터,None,극히 적음,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),종명: 고라니 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,None
8,1515,갈사만진입도로3호선 생태이동로,경상남도 하동군 금성면 가덕리 산173-16번지 일원,갈사만진입도로,3호선,왕복4차선,34,58.0,38.450000,127,...,경도 127도 48분 42.25초,20180326 ~ 20180327,None,"신희찬, 조현정",시설없음,극히 적음,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,높음(거의 매일 야생동물 이용),None,None
28,1035,곱든고개 생태통로,경기도 용인시 처인구 원삼면 사암리,소로1-11,57호선,왕복2차선,37,11.0,13.510000,127,...,경도 127도 17분 46.66초,2020.06.25. ~ 2020.06.25.,용인시,이정미,None,매일 다수,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),종명: 고라니 이용확인종: 1 이용빈도 순위: 1 멸종위기종 이용빈도 :,없음
30,1462,광주대구선 28.80k,전라북도 순창군 순창읍 백산리 366-1,광주대구선,제12호선,왕복4차선,35,21.0,58.200000,127,...,경도 127도 07분 26.9초,2016.01.01. ~ 2016.12.31.,한국도로공사 광주지사,장보고,None,None,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),None,"센서카메라 시인성 불량, 교체필요"
35,1181,구룡령 생태통로,강원도 양양군 서면 갈천리,국도,(56호선),왕복4차선,37,52.0,48.691681,128,...,경도 128도 30분 51.388초,2021.04.14. ~ 2021.07.15.,원주지방환경청,"박명종, 김용학, 심완섭",None,월 1~4회,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),종명: 산양 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 : 극히 적음\n...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,1135,None,전라남도 구례군 산동면 둔사리,산동-고달간,103호선,왕복2차선,35,18.0,18.520000,127,...,경도 127도 23분 42.41초,None,None,None,센서카메라(스틸),일 1~2회,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),None,None
454,1134,None,전라남도 구례군 신학리,산동고달간,103호선,왕복2차선,35,19.0,5.312109,127,...,경도 127도 25분 34.32초,None,None,None,None,일 1~2회,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),종명: 삵 이용확인종: 이용빈도 순위: 멸종위기종 이용빈도 : 월 1~4회,None
482,1107,None,충청남도 금산군 금성면 두곡리,금성,103호,왕복2차선,36,8.0,55.470000,127,...,경도 127도 27분 49.77초,20130901 ~ 20131130,금산군환경자원과,지권열외 1인,시설없음,일 1~2회,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),종명: 고라니 이용확인종: 1 이용빈도 순위: 멸종위기종 이용빈도 :\n종명: ...,"- 생태통로 내부 시설 및 토양 등의 상태는 양호하며, 무인센서카메라 1대를 설치하..."
496,1075,None,충청북도 보은군 마로면 갈전리,군도,15호선(갈전-법화),왕복2차선,36,22.0,18.130316,127,...,경도 127도 50분 32.554초,2016.10.03. ~ 2016.10.31.,보은군,김찬식,None,None,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,높음(거의 매일 야생동물 이용),종명: 고라니 이용확인종: 1 이용빈도 순위: 1 멸종위기종 이용빈도 :\n종명:...,None


In [39]:
# null => 9n개
# 판단불가 => 160개
94+160
# 생태통로 효율성 값 x => 254개
# 총 536개 생태통로중에 250제외

254

In [26]:
df[df['이용확인종,이용빈도높은종,멸종위기종이용빈도'].isnull()]

,번호,생태통로명,주소,도로명,도로번호,차선_수,위도_도,위도_분,위도_초,경도_도,...,생태통로좌표(경도),모니터링기간,수행기관,수행자,모니터링방법,등산객이용빈도,모니터링신뢰도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",개선필요사항
0,1495,가계통로,경상남도 산청군 생비량면 가계리 산279-3,일반국도,20호선,왕복4차선,35,21.0,36.68,128,...,경도 128도 3분 49.5초,2017.07.17. ~ 2017.07.28.,진주국토관리사무소,박상훈,None,극히 적음,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,판단 불가,None,None
1,1277,가대리,충청북도 단양군 가곡면 가대리,국도,59호선,왕복2차선,37,2.0,2.70,128,...,경도 128도 24분 23.8초,2016.01.01. ~ 2016.09.20.,도로관리사업소,박찬일,None,None,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,판단 불가,None,None
6,1527,갈매생태통로,경기도 경기 구리시 갈매동 산 69-183,갈매동 산 69-183,도로명미부여,왕복4차선,37,62.0,0.00,127,...,경도 127도 11분 33.0초,None,None,None,None,None,None,None,None,None
8,1515,갈사만진입도로3호선 생태이동로,경상남도 하동군 금성면 가덕리 산173-16번지 일원,갈사만진입도로,3호선,왕복4차선,34,58.0,38.45,127,...,경도 127도 48분 42.25초,20180326 ~ 20180327,None,"신희찬, 조현정",시설없음,극히 적음,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,높음(거의 매일 야생동물 이용),None,None
9,1247,갈운생태통로,경기도 양평군 청운면 갈운리,국도,6호선,왕복2차선,37,31.0,56.27,127,...,경도 127도 48분 4.72초,2016.04.22. ~ 2016.04.25.,경기도 양평군 환경관리과,전찬운,None,극히 적음,추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함,판단 불가,None,도로 횡단 배수로를 생태통로로 지정한 것으로 사실상 생태통로 역할을 수행하기 어렵다...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1073,None,충청북도 괴산군 사리면 사담리,사담선,None,왕복2차선,36,49.0,35.80,127,...,경도 127도 39분 35.8초,2016.07.27. ~ 2016.07.27.,괴산군청,이세희,시설없음,극히 적음,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,보통(주 35회 야생동물 이용),None,생태통로로서의 기능을 하고 있음. 고라니 배설물 등 사용 흔적 있음
503,1482,None,충청북도 청주시 서원구 모충동 260-17,매봉로,매봉로,왕복2차선,36,37.0,26.71,127,...,경도 127도 28분 40.23초,2016.12.01. ~ 2016.12.21.,None,생태통로 관리자,시설없음,매일 다수,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,판단 불가,None,None
504,1083,None,충청북도 청주시 서원구 성화동 658,구룡산로,52번길,왕복4차선,36,36.0,53.18,127,...,경도 127도 27분 24.01초,2016.10.01. ~ 2016.12.21.,청주시,생태통로수행자,None,매일 다수,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,판단 불가,None,None
505,1084,None,충청북도 청주시 서원구 수곡동 214-30,안뜸로,47,왕복2차선,36,36.0,51.32,127,...,경도 127도 28분 51.92초,2016.10.01. ~ 2016.12.21.,청주시,생태통로수행자,None,일 1~2회,금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음,판단 불가,None,None


In [43]:
encoder = LabelEncoder()
encoder.fit(df['생태통로효율성'])
test = df['생태통로효율성']
test = encoder.transform(test)
print(test)
print(encoder.classes_)

[3 3 3 3 2 1 4 4 1 3 3 3 0 0 0 0 0 3 2 0 4 2 4 2 2 3 4 0 1 4 1 4 4 4 4 1 2
 3 4 2 3 3 3 1 3 3 0 3 4 2 1 3 3 4 3 0 0 0 1 3 1 3 0 4 4 0 2 0 2 2 0 4 4 4
 3 1 1 0 2 0 3 4 0 1 0 3 3 0 2 2 3 3 3 3 3 3 0 4 0 4 4 4 4 0 0 1 4 3 3 3 4
 1 2 4 2 1 1 1 3 0 3 0 3 1 1 2 2 1 3 0 3 3 1 3 2 3 3 3 3 2 1 3 1 1 3 0 2 2
 4 4 1 3 1 1 1 4 4 4 4 3 3 4 3 3 3 3 2 3 2 2 2 2 3 3 1 3 4 4 2 4 4 4 3 3 2
 3 4 4 2 1 3 0 3 2 0 1 0 0 0 1 4 3 2 2 1 1 2 3 4 1 3 3 0 0 0 3 3 0 0 0 0 3
 4 4 2 3 0 3 3 1 0 0 0 0 0 0 1 3 3 0 3 4 3 0 4 0 3 0 3 1 2 3 3 3 4 1 3 3 3
 4 2 3 3 3 2 3 0 1 0 0 0 2 1 2 3 1 1 0 4 4 2 2 1 1 3 0 4 4 3 2 2 0 4 0 1 3
 1 0 3 0 1 2 1 1 3 1 3 1 4 1 0 2 2 2 2 1 1 1 1 2 3 1 1 2 4 0 0 3 3 3 0 0 1
 4 2 4 3 1 1 1 0 1 1 1 1 1 1 1 3 2 3 3 3 0 3 3 0 1 3 1 0 0 0 3 0 3 1 1 3 3
 4 0 3 4 1 0 0 4 3 4 0 3 0 3 4 3 4 0 0 4 4 4 2 4 3 3 3 3 1 3 3 3 3 2 1 4 2
 3 3 3 3 1 1 1 3 3 4 1 0 0 0 0 0 2 2 4 3 3 4 4 0 3 4 4 4 2 1 0 3 0 1 1 3 0
 3 0 3 4 4 0 0 0 1 1 1 4 4 3 2 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 3 3 3 3 2
 4 1 2 3 4 0 0 0 0 0 0 4 

In [34]:
type_encoder.classes_

array(['양서·파충류 통로', '육교형', '터널형', None], dtype=object)

In [36]:
df['모니터링신뢰도'].unique()

array(['추가 조사가 있어야 본 생태통로의 효율성 판단이 가능함',
       '금번 조사는 본 생태통로의 효율성에 대한 판단이 가능한 수준이었음', None,
       '환경영향평가 대상사업으로 도로건설에 따른 생태통로 설치',
       '망마산～대인산 간의 생태통로에서 너구리와 고라니의 분변이 확인되고 있고, 야간을 통해 통로를 이용하고 있음을 확인함.'],
      dtype=object)